In [ ]:
from darts.models import (LinearRegressionModel,
                          CatBoostModel, 
                          LightGBMModel, 
                          Prophet, 
                          RandomForest, 
                          XGBModel,
                          ARIMA,
                          AutoARIMA,
                          Theta)

# model_loaded = LinearRegressionModel.load("../model/linearRegressionModel_totalOrder_14Jun67.pkl")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from darts import TimeSeries

#file name data "date"-1 
df = pd.read_csv('../csv/Ming25Jun67_total_order.csv')
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# predict future
# make timeseries Object
from darts import TimeSeries
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'])

In [ ]:
# model_loaded.fit(series)

In [ ]:
# forecast = model_loaded.predict(7)
# forecast = forecast.pd_dataframe().reset_index()
# forecast.columns = ['Date', 'Total Order']
# forecast.tail(7)

In [ ]:
# model_loaded.save("../model/linearRegressionModel_totalOrder_16Jun67.pkl")

# Loop Predict and Save to csv

In [ ]:
from darts import TimeSeries
from darts.models import LinearRegressionModel, ExponentialSmoothing, CatBoostModel, LightGBMModel, Prophet, RandomForest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#file name data "date"-1 
df = pd.read_csv('../csv/Ming25Jun67_total_order.csv')
print(df.tail())

from darts import TimeSeries
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")




In [ ]:
# Function to replace outliers with the mean of the rest of the values
from scipy.stats import zscore
from darts import TimeSeries

def replace_outliers_with_mean(df, column_name):
    # Calculate Z-scores
    z_scores = zscore(df[column_name])
    
    # Identify outliers (using a threshold of 3 for Z-score)
    outliers = np.abs(z_scores) > 3
    
    # Calculate mean of non-outliers
    mean_non_outliers = df.loc[~outliers, column_name].mean()
    
    # Replace outliers with the mean of non-outliers
    df.loc[outliers, column_name] = mean_non_outliers
    
    return df

# Replace outliers in 'gmv' column
df_remove_out = replace_outliers_with_mean(df, 'unique_order_count')
series_remove_outlier = TimeSeries.from_dataframe(df_remove_out,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")

In [ ]:
series = series_remove_outlier

In [ ]:
from darts.dataprocessing.transformers import (Scaler,)

scaler = Scaler()
series_rescaled = scaler.fit_transform(series)

In [ ]:
series_rescaled.head()

In [ ]:

# Import module
import pickle

# loaded_model = pickle.load(open("", 'rb'))

def load_model_predict(path: str, timeseries, name: str):
    loaded_model = pickle.load(open(path, 'rb'))
    loaded_model.fit(timeseries)
    loaded_model.save(path)

    forecast = loaded_model.predict(7)
    forecast  = scaler.inverse_transform(forecast)
    forecast = forecast.pd_dataframe().reset_index()
    forecast.columns = ['Date', 'Total Order']

    print(name)
    print(forecast)
    print()

    #!!! check date
    date = "2024-06-25"
    forecast.tail(7).to_csv(f'../forecast/{date}_{name}.csv', index=False)
    return forecast.tail(7)


import os
# Assign directory
directory = r"D:\Ming\ProgramLang_And_IDE\Python67\pythonEnv_darts\model"
out_df = pd.DataFrame()
first_col_date = True
# Iterate over files in directory
for name in os.listdir(directory):

    # Get the full file path
    file_path = os.path.join(directory, name)

    # Print the file path
    print(file_path)
    predict_df= load_model_predict(file_path, series_rescaled, name)
    if first_col_date: 
        out_df["Date"] = predict_df["Date"];first_col_date=False
    out_df[name] = predict_df["Total Order"]

date = "2024-06-25"
out_df.tail(7).to_csv(f'../forecast/{date}.csv', index=False)
 